
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


train = pd.read_csv('Data/train.csv',sep=',')
test = pd.read_csv('Data/test.csv', sep=',')


train.isnull().sum()
test.isnull().sum()

In [61]:
def splitDate(x, date_part):
    if(date_part == "year"):
        y = datetime.strptime(x, '%m/%d/%Y %H:%M').year
    elif(date_part == "month"):
        y = datetime.strptime(x, '%m/%d/%Y %H:%M').month
    elif(date_part == "day"):
        y = datetime.strptime(x, '%m/%d/%Y %H:%M').day
    elif(date_part == "hour"):
        y = datetime.strptime(x, '%m/%d/%Y %H:%M').hour
    return int(y)

train['year']= train['data'].apply(lambda x: splitDate(x, "year"))
train['month']= train['data'].apply(lambda x: splitDate(x, "month"))
train['day_']= train['data'].apply(lambda x: splitDate(x, "day"))
train['hour']= train['data'].apply(lambda x: splitDate(x, "hour"))

test['year']= test['data'].apply(lambda x: splitDate(x, "year"))
test['month']= test['data'].apply(lambda x: splitDate(x, "month"))
test['day_']= test['data'].apply(lambda x: splitDate(x, "day"))
test['hour']= test['data'].apply(lambda x: splitDate(x, "hour"))

In [62]:
X_train = train
X_train = X_train.drop("kwh",1)
X_train = X_train.drop("data",1)
y_train = train.loc[:,'kwh']

In [63]:
X_test = test.drop("data",1)

In [64]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X_train, y_train, test_size=0.2, random_state=0)  

#reg = LinearRegression(copy_X=True, fit_intercept=True, normalize=True).fit(X_train, y_train)
#reg.score(X_train, y_train)

def zeroAssing(x):
    if(x>=0):
        return x
    else:
        return 0

array_coef_squared_error = []

reg = LinearRegression(copy_X=True, fit_intercept=True, normalize=True).fit(X1_train, y1_train)
y_pred = reg0.predict(X1_test)
pred_d = {'Id': X1_test['id'] , 'Predicted': y_pred}
predicted_data = pd.DataFrame(data=pred_d)
predicted_data['Predicted'] = predicted_data['Predicted'].apply(lambda x: zeroAssing(x))  
array_coef_squared_error.append({"linear regression",reg.score(X1_test, y1_test), mean_squared_error(y1_test,predicted_data['Predicted'])})



tab_estimators = [10 , 50, 100, 200]
for i in tab_estimators:
    reg = RandomForestRegressor(max_depth=2,n_estimators=i).fit(X1_train, y1_train)
    print(reg.score(X1_train, y1_train))
    y_pred = reg.predict(X1_test)
    print(mean_squared_error(y1_test,y_pred))
    array_coef_squared_error.append({"random forest"+str(i),reg.score(X1_train, y1_train), mean_squared_error(y1_test,y_pred)})

pred_d = {'Id': X1_test['id'] , 'Predicted': y_pred}
predicted_data = pd.DataFrame(data=pred_d)


In [72]:
##
y_pred = reg.predict(X_test)
pred_d = {'Id': X_test['id'] , 'Predicted': y_pred}
predicted_data = pd.DataFrame(data=pred_d)
    

In [75]:
print(array_coef_squared_error)

[{0.8108643962687644, 'linear regression', 0.00805511545416806}, {'random forest10', 0.7565387730918853, 0.010805898043822884}, {'random forest50', 0.7563686139737543, 0.010812090729478663}, {0.7561937638890232, 'random forest100', 0.01081690325693965}, {0.7563003185452174, 'random forest200', 0.010813739380334794}]


In [73]:
predicted_data.to_csv('Data/submission.csv',sep=',' ,index = False)

In [74]:
#predicted_data['Predicted'] = predicted_data['Predicted'].apply(lambda x: x if x>= 0 else print(x))
#y_test.tail(10)
#predicted_data.loc[predicted_data['Id'] == 75940 ]

